<a href="https://colab.research.google.com/github/edermartelinho/Modelos_Linguagem_Neurais-LLM-/blob/main/Toxicidade_e_Vi%C3%A9s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avaliando Toxicidade e Vieses em LLMs

Apesar das grandes capacidades emergentes dos LLMs, esses ainda são modelos estatíscos que reproduzem os padrões de texto vistos em treinamento, inclusive aqueles que podem ser vistos como ofensivo, obsceno e/ou discriminatório. Com isso, ao utilizar esses modelos em aplicações e produtos, os mesmos podem disseminar linguagem ofensiva, desinformação e vieses aos seus usuários.

É importante lembrar que LLMs apenas reproduzem textos com características (impressionantemente) semelhantes a textos humanos, porém não possuem nenhum entendimento real do que estão comunicando.

In [ ]:
%%capture
!pip install transformers evaluate datasets

In [ ]:
from datasets import load_dataset
import evaluate
from transformers import pipeline
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random
import torch

# Verificando se temos acesso a uma GPU
if torch.cuda.is_available():
   dev = "cuda:0"
else:
   dev = "cpu"
device = torch.device(dev)

#  Toxicidade

Uma das características indesejadas nos textos gerados por LLMs é a toxicidade. A definição formal do termo ainda é discutida na área de Machine Learning, Ética e Jurídica por depender de diferentes contextos sociais, culturais e morais. De maneira simples, um texto tóxico é aquele que "contém um comentário rude, desrepeitoso ou insensato que faria uma pessoa sair de uma conversa" [[Perspective AI](https://perspectiveapi.com/)].

## Real Toxicity Prompts

Visando a operacionalização da avaliação de toxicidade na geração de texto de LLMs, o dataset RealToxicityPrompts (RTP) foi criado. O RTP contém diversos textos para uso como prompt em LLMs. Os prompts são divididos em níveis de toxicidade, visando avaliar o quão propenso o modelo é em gerar textos tóxicos a partir de entradas com diferentes níveis de toxicidade.

In [ ]:
# Carregando o dataset do Hugging Face
rtp_dataset = load_dataset("allenai/real-toxicity-prompts", split="train").shuffle(seed=42)[:20000]

rtp_dataset['prompt'][0]

A avaliação da toxicidade do RTP foi realizada com a API da [Perspective AI](https://perspectiveapi.com/). Nesse notebook não utilizaremos essa API e sim o modelo de classificação [Unbiased RoBERTa](https://huggingface.co/unitary/unbiased-toxic-roberta), treinado em datasets específicos para detecção de textos tóxicos, ofensivos e obscenos. Por isso, vamos separar somente o texto dos prompts do dataset, descartando os valores de toxicidade trazidas pelo dataset, e reavaliar a toxicidade.

In [ ]:
# Extraindo somente o texto dos prompts
toxic_prompts = [prompt['text'] for prompt in rtp_dataset['prompt']]

In [ ]:
# Testando o modelo de avaliação de toxicidade
toxicity = pipeline(model='unitary/unbiased-toxic-roberta', device=device)

toxicity('LLMs can reproduce toxic behaviour when prompted with toxic text', top_k=None)

## Utilizando um modelo

Para geração das continuações dos prompts iremos utilizar o pipeline do Hugging Face com a tarefa 'text-generation', a qual por padrão utiliza o modelo [GPT-2](https://huggingface.co/gpt2).

In [ ]:
llm = pipeline('text-generation', device=device)
llm('Some texts are', max_length=50, pad_token_id=50256, do_sample=False)

In [ ]:
llm.tokenizer.pad_token_id = 50256 # Definindo manualmente o token especial de 'acolchoamento' para permitir a execução em batches
continuations = llm(toxic_prompts, do_sample=False, max_new_tokens=20, pad_token_id=50256, batch_size=256)

continuations = [continuation[0]['generated_text'] for continuation in continuations]
completions = [continuations[i].replace(toxic_prompts[i], '') for i in range(len(continuations))]

Com as continuações geradas, vamos usar o Unbiased RoBERTa para atribuir um valor de toxicidade entre 0 e 1 a cada um dos prompts e continuações geradas. O valor de toxicidade pode ser visto como qual a probabilidade de uma pessoa considerar o texto como tóxico (rude, desreipeitoso ou insensato)

In [ ]:
prompts_toxicity = toxicity(toxic_prompts, batch_size=1024, top_k=None)
continuations_toxicity = toxicity(completions, batch_size=1024, top_k=None)

In [ ]:
prompts_toxicity = [item[0]['score'] for item in prompts_toxicity]
continuations_toxicity = [item[0]['score'] for item in continuations_toxicity]

Uma forma interessante de analisar o resultado dessa avaliação é verificar a propensão do modelo em repercutir o nível de toxicidade vista no prompt de entrada. Para isso, vamos dividir os exemplos em 10 conjuntos com base na toxicidade do prompt de entrada. Cada conjunto é uma lista dos valores de toxicidade dos textos gerados para prompts que tenham a toxicidade dentro de uma certa faixa de valores. Essas faixas são: $0 <= x < 0.1$; $0.1 <= x < 0.2$; ...; $0.9<= x < 1$.

In [ ]:
buckets = [[] for i in range(10)]

for i, toxicity_value in enumerate(prompts_toxicity):
    continuation_toxicity = continuations_toxicity[i]
    idx = int(10 * toxicity_value)
    buckets[idx].append(continuation_toxicity)

Agora para cada faixa vamos definir a média do valor de toxicidade dentre as continuações geradas. Com isso, podemos traçar um gráfico que mostre a relação entre o nível de toxicidade esperado na continuação feita pelo modelo, dado o nível de toxicidade do texto usado como prompt.

In [ ]:
expected_toxicity = [sum(bucket) / len(bucket) for bucket in buckets]

In [ ]:
fig, ax = plt.subplots()
x_vec = [0.05+i*0.1 for i in range(10)]

ax.plot(x_vec, expected_toxicity, label='GPT2', marker='o', linewidth=2)

ax.set(ylim=[0, 0.6], xticks=[0.1*i for i in range(10)])
ax.grid()
ax.set_ylabel("Probabilidade de Toxicidade na Continuação", fontsize=15)
ax.set_xlabel("Probabilidade de Toxicidade no prompt", fontsize=15)
ax.legend(loc=2, fontsize=20)
fig.set_size_inches(12,6)
plt.show()

#  Vieses

Outro característica indesejada nos textos de LLMs e altamente relacionada aos padrões de linguagem presentes nos dados de treinamento são os vieses. Existem diferentes maneiras pelos quais os vieses podem se manifestar nos textos. De manira geral, os LLMs apresentam vieses na geração desproporcional de textos negativos, injustos ou estereotipados contra um grupo de pessoas ou ideia [[BOLD](https://arxiv.org/pdf/2101.11718.pdf)]. É importante observar que essa métrica não se precupa com a proporção geral de toxicidade gerada pelo modelo, mas sim que a probabilidade de geração de textos desrepeitosos e positivos sejam iguais para todos os grupos. Dessa forma, mesmo um modelo que gere textos ofensivos 90% das vezes, mas de forma igual para diferentes grupos (como cristãos, mulçumanos, ateus, etc.) teria uma boa métrica de viés, conforme a definição anterior.

Vamos utilizar o dataset [BOLD](https://huggingface.co/datasets/AlexaAI/bold), o qual consiste em diversos prompts separados por grupos para analise de viés em modelos de linguagem.

In [ ]:
bold_dataset = load_dataset("AlexaAI/bold", split="train")

Nesse exemplo vamos comparar os textos gerados para prompts sobre atores e atrizes.

In [ ]:
# Filtrando os exemplos sobre os grupos 'atores' e 'atrizes'
male_prompts = bold_dataset.filter(lambda x: x['category'] == 'American_actors').shuffle(seed=42)[:500]
female_prompts = bold_dataset.filter(lambda x: x['category'] == 'American_actresses').shuffle(seed=42)[:500]

male_prompts = [p[0] for p in male_prompts['prompts']]
female_prompts = [p[0] for p in female_prompts['prompts']]

In [ ]:
len(male_prompts), len(female_prompts)

Novamente utilizaremos o pipeline de 'text-generation' da Hugging Face com o modelo padrão GPT 2 para gerar as continuações dos prompts.

In [ ]:
llm = pipeline('text-generation', device=device)
llm.tokenizer.pad_token_id = 50256

generated_female = llm(female_prompts, do_sample=False, max_new_tokens=20, batch_size=100)
generated_female = [gen[0]['generated_text'] for gen in generated_female]
generated_female = [generated_female[i].replace(female_prompts[i], '') for i in range(len(generated_female))]

generated_male = llm(male_prompts, do_sample=False, max_new_tokens=20, batch_size=100)
generated_male = [gen[0]['generated_text'] for gen in generated_male]
generated_male = [generated_male[i].replace(male_prompts[i], '') for i in range(len(generated_male))]

Vamos avaliar a diferença na polaridade dos textos gerados para cada grupo. A polaridade mensura o sentimento do texto em negativo, neutro ou positivo. A partir da diferença entre a proporção de textos em cada categoria de polaridade, podemos observar a presença ou não de viés entre os grupos analisados.

In [ ]:
regard = evaluate.load('regard', module_type='measurement')

female_eval = regard.compute(data=generated_female)
male_eval = regard.compute(data=generated_male)

A partir dos valores de polaridade dos textos gerados, calculamos a porcentagem de textos que foram classificados como negativo, neutro ou positivo para cada grupo

In [ ]:
female_count = {'positive': 0, 'neutral': 0, 'negative': 0}
for eval in female_eval['regard']:
    best_score = {'score': 0}
    for score in eval:
        if score['score'] > best_score['score'] and score['label'] != 'other':
            best_score = score
    female_count[best_score['label']] += 1 / len(female_eval['regard'])

male_count = {'positive': 0, 'neutral': 0, 'negative': 0}
for eval in male_eval['regard']:
    best_score = {'score': 0}
    for score in eval:
        if score['score'] > best_score['score'] and score['label'] != 'other':
            best_score = score
    male_count[best_score['label']] += 1 / len(male_eval['regard'])

In [ ]:
female_count, male_count

In [ ]:
fig, ax = plt.subplots()

legend_handles = []
for i, eval in enumerate([male_count, female_count]):
    neg = eval['negative']
    neut = eval['neutral']
    pos = eval['positive']

    handle = ax.bar(i, neg, color='tab:red')
    ax.bar(i, neut, color='lightsteelblue', bottom=neg)
    ax.bar(i, pos, color='tab:green', bottom=neg + neut)

    legend_handles.append(handle)

ax.set_ylabel("Proporção de Textos Gerados", fontsize=15)
ax.legend(labels=['Negative', 'Neutral', 'Positive'], fontsize=15, bbox_to_anchor=(1.6, 1), borderaxespad=0)
plt.xticks(ticks=[0, 1], labels=['Male', 'Female'], fontsize=12)
plt.yticks(ticks=[0.1*i for i in range(11)])
fig.set_size_inches(4,6)
plt.show()

O gráfico acima apresenta a comparação entre as proporções de textos negativos, neutros e positivos gerados a partir de prompts sobre atores e atrizes. Podemos observar que a proporção de textos negativos é ligeiramente maior para atrizes do que atores, apresentando um pequeno viés por parte do modelo.